# Web Scraping with Python

- Ryan Mitchell

## About non-JavaScript objects 
- Using urllib, requests and bs4 packages

# Contents

1. 기초
  - 1.1 스크래핑 기초
  - 1.2 고급 HTML 파싱
  - 1.3 크롤링 
  
2. 고급
  - 2.1 폼과 로그인 크롤링
  - 2.2 이미지 및 텍스트 인식
  - 2.3 기타

# 1. 기초

이 장에서는 다음과 같은 '웹 스크래퍼'를 말하면 떠올리는 기본적인 부분을 다룬다.

- 해당 도메인에서 HTML 데이터를 받음
- 타겟 정보를 위해 HTML 데이터 파싱
- 타겟 정보를 저장함
- (추가적으로) 위의 프로세스를 반복하기 위해 다른 페이지로 이동

## 1.1 스크래핑 기초

스크래핑을 시작할 때, 먼저 웹 브라우저가 해결해 주던 사소한 모든 것을 다시 체크하는 것으로 시작한다. HTML 포매팅, CSS 스타일링, JavaScript 실행 그리고 이미지 렌더링 없이 처음에는 매우 복잡해 보이지만 브라우저의 도움 없이 어떻게 포매팅 되고 해석되는지 이해한다면 스크래핑의 많은 부분에서 진전을 이룰 수 있다.

### 코넥션 Connection 

먼저 한 연결의 예를 들어보자.

₩1. 밥의 컴퓨터가 고, 저 전력으로 나타나는 0과 1의 스트림을 보낸다. 이 비트는 헤더와 바디를 가진 어떤 정보를 구성한다. 헤더는 밥의 로컬 라우터인 MAC 주소와 최종 목적지인 앨리스의 IP를 가지고 있다. 바디는 앨리스의 서버 애플리케이션에 요청할 내용을 담고 있다.

```python
requests.get(url, params=None, headers=None, cookies=None, auth=None, timeout=None)
```

In [31]:
from uuid import getnode as get_mac
get_mac()

31021392296862

In [34]:
import requests
sess = requests.Session()
sess.get("https://www.naver.com")

<Response [200]>

In [54]:
for k in sess.headers.keys():
    print(k, ':', sess.headers.get(k))
sess.close()

User-Agent : python-requests/2.18.4
Accept-Encoding : gzip, deflate
Accept : */*
Connection : keep-alive


In [21]:
response = requests.get("https://www.naver.com")
for k in response.headers.keys():
    print(k, ':', response.headers.get(k))

Server : NWS
Date : Sun, 15 Apr 2018 12:52:51 GMT
Content-Type : text/html; charset=UTF-8
Transfer-Encoding : chunked
Connection : keep-alive
Set-Cookie : PM_CK_rcode=09440114; expires=Sun, 15 Apr 2018 13:52:51 GMT; path=/;
Cache-Control : no-cache, no-store, must-revalidate
Pragma : no-cache
P3P : CP="CAO DSP CURa ADMa TAIa PSAa OUR LAW STP PHY ONL UNI PUR FIN COM NAV INT DEM STA PRE"
X-Frame-Options : SAMEORIGIN
Content-Encoding : gzip
Strict-Transport-Security : max-age=31536000; preload
Referrer-Policy : unsafe-url


₩2. 밥의 로컬 라우터는 밥의 MAC주소로부터 모든 0과 1의 데이터를 받아 패킷으로 재해석하고 앨리스의 IP를 향하도록 한다. 밥의 라우터는 패킷에 "from" IP로 자신의 IP를 기록하고 인터넷에 보낸다.

₩3. 밥의 패킷은 앨리스의 IP로 향하게 하는 몇몇 중간의 서버를 거친다.

₩4. 앨리스의 서버는 그녀의 IP 주소로 패킷을 받는다.

₩5. 앨리스의 서버는 패킷의 헤더에 있는 목적지 포트 번호(대부분 웹의 경우 80번)를 읽고 적절한 애플리케이션에 보낸다(웹 서버 애플리케이션)

₩6. 웹 서버 애플리케이션은 서버 프로세서로부터 다음과 같은 데이터를 받는다:
  - 이것은 GET 리퀘스트 입니다.
  - 다음과 같은 파일이 요청되었습니다: index.html

₩7. 웹 서버는 정확한 HTML 파일을 가져와 밥에게 보내기 위해 새로운 패킷으로 만듭니다. 위와 동일한 프로세스로 밥의 기계로 운송되기 위해 로컬 라우터를 통해 보냅니다.

> 이 과정에 웹 브라우저가 들어옵니까?

아닙니다. 웹 브라우저는 인터넷 역사에서 최근 개발된 것입니다(1990년).

In [56]:
from urllib.request import urlopen
html = urlopen("http://pythonscraping.com/pages/page1.html") 
print(html.read())

b'<html>\n<head>\n<title>A Useful Page</title>\n</head>\n<body>\n<h1>An Interesting Title</h1>\n<div>\nLorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n</div>\n</body>\n</html>\n'


### BeautifulSoup

- 뷰티풀숩은 XML 구조를 나타내며 쉽게 이동할 수 있는 파이썬 객체를 제공하고 복잡한 HTML을 구조화하고 정리해줍니다.

In [60]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com/pages/page1.html") 
bsObj = BeautifulSoup(html.read(), "html.parser")
print(bsObj.h1)

<h1>An Interesting Title</h1>


> 태그단위로 객체화하여 다음과 같은 메써드와 어트리뷰트를 가집니다.

In [63]:
for k in bsObj.__dict__.keys():
    print(k)

builder
is_xml
known_xml
parse_only
markup
original_encoding
declared_html_encoding
contains_replacement_characters
parser_class
name
namespace
prefix
preserve_whitespace_tags
attrs
contents
parent
previous_element
next_element
next_sibling
previous_sibling
hidden
can_be_empty_element
current_data
currentTag
tagStack
preserve_whitespace_tag_stack
_most_recent_element


In [90]:
bsObj.htmm

In [91]:
bsObj.html

<html>
<head>
<title>A Useful Page</title>
</head>
<body>
<h1>An Interesting Title</h1>
<div>
Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
</div>
</body>
</html>

### 안정적으로 코넥팅하자

- 요청 및 파싱 시 발생할 수 있는 에러를 처리하여 안정적으로 스크래핑 하도록 하자.

> 특정 도메인 요청 시

In [92]:
from urllib.request import urlopen
html = urlopen("http://www.pythonscraping.com/pages/page1.html")

- 서버에서 페이지가 발견되지 않음(404 Page Not Found)
- 서버가 발견되지 않음(500 Internal Server Error)

> 파싱 시

```python
AttributeError: 'NoneType' object has no attribute 'someTag'
```

In [97]:
from urllib.request import urlopen 
from urllib.error import HTTPError 
from bs4 import BeautifulSoup

def getTitle(url): 
    try:
        html = urlopen(url) 
    except HTTPError as e:
        return None 
    try:
        bsObj = BeautifulSoup(html.read(), "html.parser")
        title = bsObj.body.h1 
    except AttributeError as e:
        return None 
    return title


title = getTitle("http://www.pythonscraping.com/pages/page1.html") 
if title == None:
    print("Title could not be found") 
else:
    print(title)

<h1>An Interesting Title</h1>


## 1.2 고급 HTML 파싱

In [100]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com/pages/warandpeace.html") 
bsObj = BeautifulSoup(html.read(), "html.parser")

> find 와 find_all 메써드를 사용하여 다양한 파싱을 시도할 수 있음

```python
findAll(tag, attributes, recursive, text, limit, keywords)
find(tag, attributes, recursive, text, keywords)
```

- tag에 세트를 넣어 해당 태그를 필터링 가능함

In [103]:
bsObj.find_all({'h1', 'h2', 'h3', 'h4', 'h5', 'h6'})

[<h1>War and Peace</h1>, <h2>Chapter 1</h2>]

- 어트리뷰트를 통해 필터링 가능함

```python
.find_all("span", {"class":"green", "class":"red"})
```

- recursive는 디폴트가 True, False 시 객체의 children만 탐색

- text를 통해 필터링 가능함

In [105]:
import requests
from bs4 import BeautifulSoup
res = requests.get('http://mutnam.com/product/detail.html?product_no=31114&cate_no=1&display_group=3')
bsObj = BeautifulSoup(res.content, "html.parser")

- text세팅과 넥스트 엘리먼트 조합을 통해 타겟 정보의 순서가 바뀌어도 정확히 추출

In [106]:
price_prev = bsObj.find(text='판매가')

In [113]:
list(price_prev.next_elements)[1].text

'22,800원 '

- limit은 추출 개수, find_all에만 존재

- keyword는 모두 어트리뷰트를 통해 가능

```python
bsObj.findAll(id="text") # 키워드
bsObj.findAll("", {"id":"text"}) # 어트리뷰트
``` 


### BeautifulSoup objects

- BeautifulSoup object
- Tag object
- Navigablestring object
- Comment object

In [117]:
markup = "<b><!--Hey, buddy. Want to buy a used parser?--></b>"
soup = BeautifulSoup(markup, "html.parser")
comment = soup.b.string
print('type', type(comment))
print(comment)

type <class 'bs4.element.Comment'>
Hey, buddy. Want to buy a used parser?


### Navigating Trees


- 현재까지 아래와 같이 일방향으로 네비게이팅 트리를 사용했으나:

```python
bsObj.tag.subTag.anotherSubTag
```

- 위, 아래, 사선으로 이동가능

- 어떤 태그의 바로 아래 있는 태그들 : children

In [121]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com/pages/page3.html")
bsObj = BeautifulSoup(html, "html.parser")
for child in bsObj.find("table",{"id":"giftList"}).children:
    print(child)



<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>


<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>


- 해당 태그 한 단계 위의 태그 : parent

In [123]:
for parent in bsObj.find("table",{"id":"giftList"}).parent:
    print(parent)



<img src="../img/gifts/logo.jpg" style="float:left;"/>


<h1>Totally Normal Gifts</h1>


<div id="content">Here is a collection of totally normal, totally reasonable gifts that your friends are sure to love! Our collection is
hand-curated by well-paid, free-range Tibetan monks.<p>
We haven't figured out how to make online shopping carts yet, but you can send us a check to:<br/>
123 Main St.<br/>
Abuja, Nigeria
We will then send your totally amazing gift, pronto! Please include an extra $5.00 for gift wrapping.</p></div>


<table id="giftList">
<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>
<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>
<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
H